In [1]:
import pandas as pd
import os
import torch
import numpy as np
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LongformerForSequenceClassification, LongformerTokenizerFast
from transformers import LongformerTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
import time
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_json('/kaggle/input/action-data-0307-final/action_240307_sequence.json')

In [3]:
# data split function
def split_data(data, sample_SEED, fixSEED=0):
  # normal bot split
  normal = data[data['restrict']==0].reset_index(drop=True)
  bot = data[data['restrict']==1].reset_index(drop=True)
  # normal underampling
  random.seed(sample_SEED)
  sample = random.sample(list(range(len(normal))), len(bot))
  normal = normal.loc[sample].reset_index(drop=True)
  data = pd.concat([normal, bot], axis=0)
  data['restrict'].value_counts()
  # train valid test split
  train, test = train_test_split(data, test_size=0.3, random_state=fixSEED, stratify = data['restrict'])
  train, valid = train_test_split(train, test_size=0.3, random_state=fixSEED, stratify = train['restrict'])
  train = train.reset_index(drop=True)
  valid = valid.reset_index(drop=True)
  test = test.reset_index(drop=True)
  # 1:1 fix
  random.seed(fixSEED)
  move = random.sample(list(valid[valid['restrict']==1].index), 1)
  test = pd.concat([test, valid.loc[move]], axis=0).reset_index(drop=True)
  valid = valid.drop(move, axis=0).reset_index(drop=True)
  return train, valid, test

def extract_first_two_sentences(text):
    sentences = re.split(r'(?<=\.)\s+', text)
    return ' '.join(sentences[:2])

# SEED = 5

In [4]:
# data split function
def split_data(data, sample_SEED, fixSEED=0):
  # normal bot split
  normal = data[data['restrict']==0].reset_index(drop=True)
  bot = data[data['restrict']==1].reset_index(drop=True)
  # normal underampling
  random.seed(sample_SEED)
  sample = random.sample(list(range(len(normal))), len(bot))
  normal = normal.loc[sample].reset_index(drop=True)
  data = pd.concat([normal, bot], axis=0)
  data['restrict'].value_counts()
  # train valid test split
  train, test = train_test_split(data, test_size=0.3, random_state=fixSEED, stratify = data['restrict'])
  train, valid = train_test_split(train, test_size=0.3, random_state=fixSEED, stratify = train['restrict'])
  train = train.reset_index(drop=True)
  valid = valid.reset_index(drop=True)
  test = test.reset_index(drop=True)
  # 1:1 fix
  random.seed(fixSEED)
  move = random.sample(list(valid[valid['restrict']==1].index), 1)
  test = pd.concat([test, valid.loc[move]], axis=0).reset_index(drop=True)
  valid = valid.drop(move, axis=0).reset_index(drop=True)
  return train, valid, test

def extract_first_two_sentences(text):
    sentences = re.split(r'(?<=\.)\s+', text)
    return ' '.join(sentences[:2])


# train valid test split
df_train, df_valid, df_test = split_data(data, 5)

# Apply the function to the text_column and create a new column
df_train['base'] = df_train['longlongtext'].apply(extract_first_two_sentences)
df_valid['base'] = df_valid['longlongtext'].apply(extract_first_two_sentences)
df_test['base'] = df_test['longlongtext'].apply(extract_first_two_sentences)

In [5]:
df_train

,base_ymd,playerid,text,longtext,longlongtext,restrict,base
0,2024-03-07,3a205556d558613d4042345066961b67abee8eee0bb3bb...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 1...,"The count of unique action type is 7. Maximum,...",0,"The count of unique action type is 7. Maximum,..."
1,2024-03-07,0487d4e85bcb1a95daab158ec1f74cec19208862d690d9...,The user performed the action 'item collection...,The maximum number of consecutive actions is 1...,"The count of unique action type is 8. Maximum,...",1,"The count of unique action type is 8. Maximum,..."
2,2024-03-07,8d79d204afbc4dae19451aa064ad6fc8b7974630dd2830...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 4...,"The count of unique action type is 2. Maximum,...",1,"The count of unique action type is 2. Maximum,..."
3,2024-03-07,4fe53a78282425ab8d06c9a9f4802a4c421c2a6c072790...,The user performed the action 'stat_set_monste...,The maximum number of consecutive actions is 3...,The count of unique action type is 12. Maximum...,1,The count of unique action type is 12. Maximum...
4,2024-03-07,c7be3a2578338258c2d06681da01859423801e6ffbc36c...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 7...,"The count of unique action type is 7. Maximum,...",0,"The count of unique action type is 7. Maximum,..."
...,...,...,...,...,...,...,...
5637,2024-03-07,23d7d6896692f1e04ea407c896fdb78cfc2d02a0179bf0...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 1...,The count of unique action type is 10. Maximum...,0,The count of unique action type is 10. Maximum...
5638,2024-03-07,62b0c9cd10e22ea5ed2206f9de9acf9a5efb73428a4961...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 8...,"The count of unique action type is 4. Maximum,...",1,"The count of unique action type is 4. Maximum,..."
5639,2024-03-07,c34c077635dffbc156483f491eba041fa794c388a8a1b5...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 6...,"The count of unique action type is 5. Maximum,...",0,"The count of unique action type is 5. Maximum,..."
5640,2024-03-07,f19e7c0d44726ce3a075d8c2275b59d5f16e3041414e07...,The user performed the action 'warp_teleport_n...,The maximum number of consecutive actions is 8...,"The count of unique action type is 7. Maximum,...",1,"The count of unique action type is 7. Maximum,..."


In [30]:
# Longformer model, tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

In [4]:
# Seed 설정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Training function
def train(model, train_loader, optimizer, scheduler):
    model.train()
    total_loss = 0
    scaler = torch.cuda.amp.GradScaler()

    for step, batch in enumerate(tqdm(train_loader, desc="Training")):
        with torch.cuda.amp.autocast():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['restrict'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / gradient_accumulation_steps
            total_loss += loss.item()

        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()

    return total_loss / len(train_loader)

# Evaluation function
def evaluate(model, data_loader):
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['restrict'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, axis=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return preds, true_labels



In [5]:
# Tokenization function
def tokenize_function(examples, key, max_length):
    return tokenizer(examples[key], padding='max_length', truncation=True, max_length=max_length)

# Preprocessing function
def prepare_data(df, key, max_length):
    data = {key: df[key].tolist(), 'restrict': df['restrict'].tolist()}
    dataset = Dataset.from_dict(data)
    dataset = dataset.map(lambda x: tokenize_function(x, key, max_length), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'restrict'])
    return dataset

# Dataloader function
def dataloader(df_train, df_valid, df_test, variable):
    # Define the variable-to-key mapping
    key_map = {
        'text': ('text', 1024),
        'longtext': ('longtext', 1024),
        'longlongtext': ('longlongtext', 1024),
        'base': ('base', 31)
    }
    
    # Retrieve the appropriate key and max_length for the variable
    key, max_length = key_map[variable]
    
    # Prepare datasets
    train_dataset = prepare_data(df_train, key, max_length)
    val_dataset = prepare_data(df_valid, key, max_length)
    test_dataset = prepare_data(df_test, key, max_length)
    
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1)
    test_loader = DataLoader(test_dataset, batch_size=1)
    
    return train_loader, val_loader, test_loader



## Text5

In [38]:
train_loader, val_loader, test_loader = dataloader(df_train, df_valid, df_test, 'text')


# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 2

# Learning rate scheduler
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
gradient_accumulation_steps = 8
seed = 0
set_seed(seed)

Map:   0%|          | 0/5642 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

In [39]:
num_epochs = 2

seed = 0
set_seed(seed)

# 훈련 시간 측정 시작
start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    train_loss = train(model, train_loader, optimizer, scheduler)
    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}, Loss: {train_loss}")
    print(f"Epoch {epoch + 1} Training Time: {epoch_end_time - epoch_start_time:.2f} seconds")

    # validation data 평가
    predicted_labels, true_labels = evaluate(model, val_loader)
    f1 = f1_score(true_labels, predicted_labels, zero_division=1)
    precision = precision_score(true_labels, predicted_labels, zero_division=1)
    recall = recall_score(true_labels, predicted_labels, zero_division=1)
    accuracy = accuracy_score(true_labels, predicted_labels)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    print(f"Epoch {epoch + 1}, Validation F1 Score: {f1}")
    print(f"Validation Precision: {precision}")
    print(f"Validation Recall: {recall}")
    print(f"Validation Accuracy: {accuracy}")
    print("Validation Confusion Matrix:\n", conf_matrix)


# 훈련 시간 측정 종료
end_time = time.time()
total_training_time = end_time - start_time # 훈련시간

# Final prediction
predicted_labels, true_labels = evaluate(model, test_loader)
f1 = f1_score(true_labels, predicted_labels, zero_division=1)
precision = precision_score(true_labels, predicted_labels, zero_division=1)
recall = recall_score(true_labels, predicted_labels, zero_division=1)
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Final Test Results:")
print(f"Test F1 Score: {f1}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test Accuracy: {accuracy}")
print("Test Confusion Matrix:\n", conf_matrix)
print(f"Total Training Time: {total_training_time:.2f} seconds")

/tmp/ipykernel_36/324861209.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training:   0%|          | 0/5642 [00:00<?, ?it/s]/tmp/ipykernel_36/324861209.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 100%|██████████| 5642/5642 [25:10<00:00,  3.73it/s]


Epoch 1, Loss: 0.08162665003709275
Epoch 1 Training Time: 1510.91 seconds


Evaluating: 100%|██████████| 2418/2418 [03:25<00:00, 11.75it/s]
/tmp/ipykernel_36/324861209.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Epoch 1, Validation F1 Score: 0.7268258426966292
Validation Precision: 0.6314826113483831
Validation Recall: 0.8560794044665012
Validation Accuracy: 0.6782464846980976
Validation Confusion Matrix:
 [[ 605  604]
 [ 174 1035]]


Training:   0%|          | 0/5642 [00:00<?, ?it/s]/tmp/ipykernel_36/324861209.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 100%|██████████| 5642/5642 [25:11<00:00,  3.73it/s]


Epoch 2, Loss: 0.07635324012299287
Epoch 2 Training Time: 1511.47 seconds


Evaluating: 100%|██████████| 2418/2418 [03:27<00:00, 11.68it/s]


Epoch 2, Validation F1 Score: 0.7290540540540541
Validation Precision: 0.6162193032552827
Validation Recall: 0.8924731182795699
Validation Accuracy: 0.6683209263854425
Validation Confusion Matrix:
 [[ 537  672]
 [ 130 1079]]


Evaluating: 100%|██████████| 3456/3456 [04:54<00:00, 11.74it/s]

Final Test Results:
Test F1 Score: 0.7303803562831006
Test Precision: 0.6253091508656224
Test Recall: 0.8778935185185185
Test Accuracy: 0.6759259259259259
Test Confusion Matrix:
 [[ 819  909]
 [ 211 1517]]
Total Training Time: 3435.31 seconds


## longtext5

In [41]:
# Longformer model, tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

train_loader, val_loader, test_loader = dataloader(df_train, df_valid, df_test, 'longtext')

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 2

# Learning rate scheduler
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
gradient_accumulation_steps = 8
seed = 0
set_seed(seed)

num_epochs = 2

# 훈련 시간 측정 시작
start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    train_loss = train(model, train_loader, optimizer, scheduler)
    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}, Loss: {train_loss}")
    print(f"Epoch {epoch + 1} Training Time: {epoch_end_time - epoch_start_time:.2f} seconds")

    # validation data 평가
    predicted_labels, true_labels = evaluate(model, val_loader)
    f1 = f1_score(true_labels, predicted_labels, zero_division=1)
    precision = precision_score(true_labels, predicted_labels, zero_division=1)
    recall = recall_score(true_labels, predicted_labels, zero_division=1)
    accuracy = accuracy_score(true_labels, predicted_labels)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    print(f"Epoch {epoch + 1}, Validation F1 Score: {f1}")
    print(f"Validation Precision: {precision}")
    print(f"Validation Recall: {recall}")
    print(f"Validation Accuracy: {accuracy}")
    print("Validation Confusion Matrix:\n", conf_matrix)


# 훈련 시간 측정 종료
end_time = time.time()
total_training_time = end_time - start_time # 훈련시간

# Final prediction
predicted_labels, true_labels = evaluate(model, test_loader)
f1 = f1_score(true_labels, predicted_labels, zero_division=1)
precision = precision_score(true_labels, predicted_labels, zero_division=1)
recall = recall_score(true_labels, predicted_labels, zero_division=1)
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Final Test Results:")
print(f"Test F1 Score: {f1}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test Accuracy: {accuracy}")
print("Test Confusion Matrix:\n", conf_matrix)
print(f"Total Training Time: {total_training_time:.2f} seconds")

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5642 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

/tmp/ipykernel_36/324861209.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training:   0%|          | 0/5642 [00:00<?, ?it/s]/tmp/ipykernel_36/324861209.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 100%|██████████| 5642/5642 [25:15<00:00,  3.72it/s]


...

Epoch 2, Loss: 0.07714815285137383
Epoch 2 Training Time: 1510.87 seconds


Evaluating: 100%|██████████| 2418/2418 [03:25<00:00, 11.75it/s]


Epoch 2, Validation F1 Score: 0.733837689133425
Validation Precision: 0.6280164802825191
Validation Recall: 0.8825475599669148
Validation Accuracy: 0.6799007444168734
Validation Confusion Matrix:
 [[ 577  632]
 [ 142 1067]]


Evaluating: 100%|██████████| 3456/3456 [04:53<00:00, 11.77it/s]

Final Test Results:
Test F1 Score: 0.7335623159960745
Test Precision: 0.6367120954003407
Test Recall: 0.8651620370370371
Test Accuracy: 0.6857638888888888
Test Confusion Matrix:
 [[ 875  853]
 [ 233 1495]]
Total Training Time: 3439.01 seconds


## longlongtext5

In [43]:
# Longformer model, tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

train_loader, val_loader, test_loader = dataloader(df_train, df_valid, df_test, 'longlongtext')

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 2

# Learning rate scheduler
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
gradient_accumulation_steps = 8
seed = 0
set_seed(seed)

num_epochs = 2

# 훈련 시간 측정 시작
start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    train_loss = train(model, train_loader, optimizer, scheduler)
    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}, Loss: {train_loss}")
    print(f"Epoch {epoch + 1} Training Time: {epoch_end_time - epoch_start_time:.2f} seconds")

    # validation data 평가
    predicted_labels, true_labels = evaluate(model, val_loader)
    f1 = f1_score(true_labels, predicted_labels, zero_division=1)
    precision = precision_score(true_labels, predicted_labels, zero_division=1)
    recall = recall_score(true_labels, predicted_labels, zero_division=1)
    accuracy = accuracy_score(true_labels, predicted_labels)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    print(f"Epoch {epoch + 1}, Validation F1 Score: {f1}")
    print(f"Validation Precision: {precision}")
    print(f"Validation Recall: {recall}")
    print(f"Validation Accuracy: {accuracy}")
    print("Validation Confusion Matrix:\n", conf_matrix)


# 훈련 시간 측정 종료
end_time = time.time()
total_training_time = end_time - start_time # 훈련시간

# Final prediction
predicted_labels, true_labels = evaluate(model, test_loader)
f1 = f1_score(true_labels, predicted_labels, zero_division=1)
precision = precision_score(true_labels, predicted_labels, zero_division=1)
recall = recall_score(true_labels, predicted_labels, zero_division=1)
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Final Test Results:")
print(f"Test F1 Score: {f1}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test Accuracy: {accuracy}")
print("Test Confusion Matrix:\n", conf_matrix)
print(f"Total Training Time: {total_training_time:.2f} seconds")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5642 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

/tmp/ipykernel_36/324861209.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training:   0%|          | 0/5642 [00:00<?, ?it/s]/tmp/ipykernel_36/324861209.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 100%|██████████| 5642/5642 [25:10<00:00,  3.74it/s]


...

Epoch 2, Loss: 0.07694968108968589
Epoch 2 Training Time: 1516.17 seconds


Evaluating: 100%|██████████| 2418/2418 [03:28<00:00, 11.62it/s]


Epoch 2, Validation F1 Score: 0.7345529858474283
Validation Precision: 0.6303317535545023
Validation Recall: 0.880066170388751
Validation Accuracy: 0.6819685690653433
Validation Confusion Matrix:
 [[ 585  624]
 [ 145 1064]]


Evaluating: 100%|██████████| 3456/3456 [04:57<00:00, 11.62it/s]

Final Test Results:
Test F1 Score: 0.734633423845964
Test Precision: 0.6405510116229014
Test Recall: 0.8611111111111112
Test Accuracy: 0.6889467592592593
Test Confusion Matrix:
 [[ 893  835]
 [ 240 1488]]
Total Training Time: 3441.29 seconds


## Base5

In [44]:
# Longformer model, tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

train_loader, val_loader, test_loader = dataloader(df_train, df_valid, df_test, 'base')

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 2

# Learning rate scheduler
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
gradient_accumulation_steps = 8
seed = 0
set_seed(seed)

num_epochs = 2

# 훈련 시간 측정 시작
start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    train_loss = train(model, train_loader, optimizer, scheduler)
    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}, Loss: {train_loss}")
    print(f"Epoch {epoch + 1} Training Time: {epoch_end_time - epoch_start_time:.2f} seconds")

    # validation data 평가
    predicted_labels, true_labels = evaluate(model, val_loader)
    f1 = f1_score(true_labels, predicted_labels, zero_division=1)
    precision = precision_score(true_labels, predicted_labels, zero_division=1)
    recall = recall_score(true_labels, predicted_labels, zero_division=1)
    accuracy = accuracy_score(true_labels, predicted_labels)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    print(f"Epoch {epoch + 1}, Validation F1 Score: {f1}")
    print(f"Validation Precision: {precision}")
    print(f"Validation Recall: {recall}")
    print(f"Validation Accuracy: {accuracy}")
    print("Validation Confusion Matrix:\n", conf_matrix)


# 훈련 시간 측정 종료
end_time = time.time()
total_training_time = end_time - start_time # 훈련시간

# Final prediction
predicted_labels, true_labels = evaluate(model, test_loader)
f1 = f1_score(true_labels, predicted_labels, zero_division=1)
precision = precision_score(true_labels, predicted_labels, zero_division=1)
recall = recall_score(true_labels, predicted_labels, zero_division=1)
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Final Test Results:")
print(f"Test F1 Score: {f1}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test Accuracy: {accuracy}")
print("Test Confusion Matrix:\n", conf_matrix)
print(f"Total Training Time: {total_training_time:.2f} seconds")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5642 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

/tmp/ipykernel_36/324861209.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training:   0%|          | 0/5642 [00:00<?, ?it/s]/tmp/ipykernel_36/324861209.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Input ids are automatically padded to be a multiple of `config.attention_window`: 512
Training: 100%|██████████| 5642/5642 [16:55<00:00,  5.56it/s]


...

Epoch 2, Loss: 0.08067244158193432
Epoch 2 Training Time: 1005.58 seconds


Evaluating: 100%|██████████| 2418/2418 [02:25<00:00, 16.60it/s]


Epoch 2, Validation F1 Score: 0.7086113759135685
Validation Precision: 0.5753353973168215
Validation Recall: 0.9222497932175352
Validation Accuracy: 0.6207609594706369
Validation Confusion Matrix:
 [[ 386  823]
 [  94 1115]]


Evaluating: 100%|██████████| 3456/3456 [03:27<00:00, 16.68it/s]

Final Test Results:
Test F1 Score: 0.7099253900067827
Test Precision: 0.5825602968460112
Test Recall: 0.9085648148148148
Test Accuracy: 0.6287615740740741
Test Confusion Matrix:
 [[ 603 1125]
 [ 158 1570]]
Total Training Time: 2312.88 seconds
